# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 20
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)



In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

120000

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AAE'
recording_dir='recording'+sep_local+model_name

In [18]:
from training.adversarial_basic.AAE import AAE as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recording\\pokemonAAE\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 32)      128       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 32)      1056      
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                40960032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [26]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                40961344  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       3962124   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 44,923,468
Trainable params: 44,923,398
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [27]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [28]:
progbar = NotebookPrograssBar(leave_outer=False)

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [30]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [31]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recording\\pokemonAAE\\csv_dir\\pokemonAAE.csv'

In [32]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [33]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [34]:
import numpy as np

# Model Training

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=750,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=250
)


training traditional basicAE


Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Model: "inference_discriminator_real"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
____________________

Epoch 9/10
10/10 [==============================] - ETA: 0s - loss: 0.190 - ETA: 0s - loss: 0.182 - ETA: 0s - loss: 0.172 - ETA: 0s - loss: 0.169 - ETA: 0s - loss: 0.167 - ETA: 0s - loss: 0.166 - 4s 447ms/step - loss: 0.1648 - val_loss: 0.0062
Epoch 10/10
10/10 [==============================] - ETA: 0s - loss: 0.160 - ETA: 0s - loss: 0.167 - ETA: 0s - loss: 0.172 - ETA: 0s - loss: 0.174 - ETA: 0s - loss: 0.178 - 4s 410ms/step - loss: 0.1811 - val_loss: 0.0109
Model: "latent_AA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_inputs (InputLayer)   [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference (Model)               (None, 32)           40961344    inference_inputs[0][0]           
_____________________

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027A9DB3B788>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027AABEF7288>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027A9DB3B788>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027AABEF7288>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
Instructions for updating:
If using Keras pass *_constraint 

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027A9DB3B788>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027AABEF7288>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027B0868D408>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x0000027A9DB3B788>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<